2024 ICPPB Sourmash LIN tutorial
===

# Analyzing Metagenome Composition using the LIN taxonomic framework

Tessa Pierce Ward

July 2024

requires sourmash v4.8+

This tutorial uses the `sourmash taxonomy` module, which was introduced via [blog post](https://bluegenes.github.io/sourmash-tax/)
and was recently shown to perfom well for taxonomic profiling of long (and short) reads in [Evaluation of taxonomic classification and profiling methods for long-read shotgun metagenomic sequencing datasets](https://link.springer.com/article/10.1186/s12859-022-05103-0), Portik et al., 2022.


In this tutorial, we'll use sourmash gather to analyze metagenomes using the [LIN taxonomic framework](https://dl.acm.org/doi/pdf/10.1145/3535508.3545546).
Specifically, we will analyze plant metagenomes for the presence of _Ralstonia solanacearum_.
The goal is to see if we can correctly assign the sequence in each file to the correct phylogenetic group, distinguishing between pathogenic and non-pathogenic strains.

**Simulated Samples: Ralstonia + tomato host:**
- `Sample0` - no Ralstonia
- `Sample-II` - Ralstonia solanacearum, PhylIIB
- `SampleIV` - Ralstonia solanacearum, Phyl-IV

**Additional Sample (nanopore):**
- `barcode16`

## Setup

In [29]:
## Download Ralstonia 32-genome database and corresponding taxonomy files

# database
!curl -JLO https://osf.io/download/wxtk3/
!mv ralstonia.sc1000.zip ralstonia.zip

# taxonomy csv
!curl -JLO https://osf.io/download/sj2z7/
!mv ralstonia32.lin-taxonomy.csv ralstonia.lin-taxonomy.csv

# lingroup csv
!curl -JLO https://osf.io/download/nqms2/
!mv LINgroups.csv ralstonia.lingroups.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377  100   377    0     0   1514      0 --:--:-- --:--:-- --:--:--  1520
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 4274k  100 4274k    0     0  2506k      0  0:00:01  0:00:01 --:--:-- 8440k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377  100   377    0     0   1673      0 --:--:-- --:--:-- --:--:--  1668     0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
curl: (23) Failed writing header
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   S

In [14]:
### Next, download pre-made sourmash signatures made from the input metagenomes

# download Sample-0 signature
!curl -JLO https://osf.io/download/dvyt9/

# download Sample-II signature
!curl -JLO https://osf.io/download/agwdu/

# download Sample-IV signature
!curl -JLO https://osf.io/download/rngjq/

# move downloaded signatures to ./inputs
!mv Sample*.zip ./inputs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377  100   377    0     0   1455      0 --:--:-- --:--:-- --:--:--  1461
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  821k  100  821k    0     0   388k      0  0:00:02  0:00:02 --:--:--  388k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377  100   377    0     0   1312      0 --:--:-- --:--:-- --:--:--  1309
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  839k  100  839k    0     0   570k      0  0:00:01  0:00:01 --:--:-- 5675k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   377  100   377    0     0   1414      0 --:--:

### Look at the signatures

Let's start with the `Sample-II` sample

By running `sourmash sig fileinfo`, we can see information on the signatures available within the zip file.

Here, you can see I've generated the metagenome signature with `scaled=100` and built three ksizes, `k=21`, k=31` and `k=51`

In [18]:
!sourmash sig fileinfo ./inputs/Sample-II.sc1000.zip


== This is sourmash version 4.8.10. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

** loading from './inputs/Sample-II.sc1000.zip'
path filetype: ZipFileLinearIndex
location: /Users/ntward/dib-lab/2024-icppb/inputs/Sample-II.sc1000.zip
is database? yes
has manifest? yes
num signatures: 3
** examining manifest...
total hashes: 105825
summary of sketches:
   1 sketches with DNA, k=21, scaled=1000, abund      33335 total hashes
   1 sketches with DNA, k=31, scaled=1000, abund      35516 total hashes
   1 sketches with DNA, k=51, scaled=1000, abund      36974 total hashes


### Look at the database

Here, you can see I've generated the database with `scaled=1000` and built three ksizes, `k=21`, `k=31` and `k=51`

In [17]:
!sourmash sig fileinfo ralstonia.zip


== This is sourmash version 4.8.10. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

** loading from 'ralstonia.zip'
path filetype: ZipFileLinearIndex
location: /Users/ntward/dib-lab/2024-icppb/ralstonia.zip
is database? yes
has manifest? yes
num signatures: 96
** examining manifest...
total hashes: 524340
summary of sketches:
   32 sketches with DNA, k=21, scaled=1000            174967 total hashes
   32 sketches with DNA, k=51, scaled=1000            174975 total hashes
   32 sketches with DNA, k=31, scaled=1000            174398 total hashes


> There's a lot of things to digest in this output but the two main ones are:
> * there are 32 genomes represented in this database, each of which are sketched at k=21,k=31,k=51
> * this database represents ~524 *million* k-mers (multiply number of hashes by the scaled number)


## Build a taxonomic profile of Sample II

First, let's run `sourmash gather` to find the closest reference genome(s) in the database.
If you want to read more about what sourmash is doing, please see [Lightweight compositional analysis of metagenomes with FracMinHash and minimum metagenome covers](https://www.biorxiv.org/content/10.1101/2022.01.11.475838v2), Irber et al., 2022.

In [21]:
!sourmash gather inputs/Sample-II.sc1000.zip \
                 ralstonia.zip -k 31 \
                --output Sample-II.k31.gather.csv


== This is sourmash version 4.8.10. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

selecting specified query k=31
loaded query: Sample-II... (k=31, DNA)
--ading from 'ralstonia.zip'...
loaded 96 total signatures from 1 locations.
after selecting signatures compatible with search, 32 remain.

Starting prefetch sweep across databases.
Prefetch found 31 signatures with overlap >= 50.0 kbp.
Doing gather to generate minimum metagenome cover.

overlap     p_query p_match avg_abund
---------   ------- ------- ---------
1.3 Mbp        3.9%   26.6%       1.2    GCF_001373295.1 Ralstonia solanacear...
found less than 50.0 kbp in common. => exiting

found 1 matches total;
the recovered matches hit 3.9% of the abundance-weighted query.
the recovered matches hit 3.6% of the query k-mers (unweighted).



> The first step of gather ("prefetch") found all potential matches with at least 50kb matching sequence (31 of 32 total database genomes). Then, the greedy algorithm narrowed this to a single best match, ` GCF_001373295.1` which shared an estimated 1.3 Mbp with the metagenome (~3.9% of the total query dataset). We can visualize this by looking at a venn diagram of the shared k-mers between metagenome sample and the top match. The yellow intersection represend <4% of the metagenome and ~26.6% of the Ralstonia RS2 reference genome. This small match percentage is expected, though, since the dataset is a simulated plant metagenome with an in silico Ralstonia spike-in, and we are just searching for `Ralstonia` here.
![sampleii.venn](https://hackmd.io/_uploads/SyssO_QPC.png)

### Add taxonomic information and summarize up lingroups

`sourmash gather` finds the smallest set of reference genomes that contains all the known information (k-mers) in the metagenome. In most cases, `gather` will find many metagenome matches. Here, we're only looking for `Ralstonia` matches and we only have a single gather result. Regardless, let's use `sourmash tax metagenome` to add taxonomic information and see if we've correctly assigned the pathogenic sequence.

#### First, let's look at the relevant taxonomy files.

These commands will show the first few lines of each file. If you prefer, you can look at a more human-friendly view by opening the files in a spreadsheet program.

- **taxonomy_csv:** `ralstonia32.lin-taxonomy.csv`
  - the essential columns are `lin` (`14;1;0;...`) and `accession` (`GCF_00`...)
- **lingroups information:** `ralstonia.lingroups.csv`
  - both columns are essential (`name`, `lin`)

Look at the taxonomy file:

In [23]:
!head -n 5 ralstonia32.lin-taxonomy.csv

lin,species,strain,filename,accession
14;1;0;0;0;0;0;0;0;0;6;0;1;0;1;0;0;0;0;0,Ralstonia solanacearum,OE1_1,GCF_001879565.1_ASM187956v1_genomic.fna,GCF_001879565.1
14;1;0;0;0;0;0;0;0;0;6;0;1;0;0;0;0;0;0;0,Ralstonia solanacearum,PSS1308,GCF_001870805.1_ASM187080v1_genomic.fna,GCF_001870805.1
14;1;0;0;0;0;0;0;0;0;2;1;0;0;0;0;0;0;0;0,Ralstonia solanacearum,FJAT_1458,GCF_001887535.1_ASM188753v1_genomic.fna,GCF_001887535.1
14;1;0;0;0;0;0;0;0;0;2;0;0;4;4;0;0;0;0;0,Ralstonia solanacearum,Pe_13,GCF_012062595.1_ASM1206259v1_genomic.fna,GCF_012062595.1


> The key columns are:
> - `accession`, containing identifiers matching the database sketches
> - `lin`, containing the LIN taxonomic information.

Now, let's look at the lingroups file:

In [26]:
!head -n5 LINgroups.csv

name,lin
A_Total_reads;B_PhylI,14;1;0;0;0;0;0;0;0;0
A_Total_reads;B_PhylI;C_seq14,14;1;0;0;0;0;0;0;0;0;3
A_Total_reads;B_PhylI;C_seq15,14;1;0;0;0;0;0;0;0;0;2
A_Total_reads;B_PhylI;C_seq34,14;1;0;0;0;0;0;0;0;0;6


> Here, we have two columns:
> - `name` - the name for each lingroup. 
> - `lin` - the LIN prefix corresponding to each group.

### Now, run `sourmash tax metagenome` to integrate taxonomic information into `gather` results

Using the `gather` output we generated above, we can integrate taxonomic information and summarize up "ranks" (lin positions). We can produce several different types of outputs, including a `lingroup` report.

`lingroup` format summarizes the taxonomic information at each `lingroup`, and produces a report with 4 columns: 
- `name` (from lingroups file)
- `lin` (from lingroups file)
- `percent_containment` - total % of the file matched to this lingroup
- `num_bp_contained` - estimated number of bp matched to this lingroup

> Since sourmash assigns all k-mers to individual genomes, no reads/base pairs are "assigned" to higher taxonomic ranks or lingroups (as with Kraken-style LCA). Here, "percent_containment" and "num_bp_contained" is calculated by summarizing the assignments made to all genomes in a lingroup. This is akin to the "contained" information in Kraken-style reports.

Run `tax metagenome`:

In [33]:
!sourmash tax metagenome -g Sample-II.k31.gather.csv \
                        -t ralstonia.lin-taxonomy.csv \
                        --lins --lingroup ralstonia.lingroups.csv


== This is sourmash version 4.8.10. ==
== Please cite Irber et. al (2024), doi:10.21105/joss.06830. ==

Trying to read LIN taxonomy assignments.
loaded 1 gather results from 'Sample-II.k31.gather.csv'.
loaded results for 1 queries from 1 gather CSVs
Read 20 lingroup rows and found 20 distinct lingroup prefixes.
name	lin	percent_containment	num_bp_contained
A_Total_reads;B_PhylII	14;1;0;0;0;3;0	3.94	1464000
A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2	14;1;0;0;0;3;0;0;0;0;1;0;0;0;0	3.94	1464000
A_Total_reads;B_PhylII;C_IIB	14;1;0;0;0;3;0;0	3.94	1464000
A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2;E_seq1	14;1;0;0;0;3;0;0;0;0;1;0;0;0;0;0;0	3.94	1464000


> Here, the most specific lingroup we assign to is `A_Total_reads;B_PhylII;C_IIB;D_seq1&seq2;E_seq1`, which means this is in **phylotype IIB, sequevar 1**. This is the USDA select agent!

## Repeat for Remaining Samples